# Neural Network - Exercise 01

In [ ]:
import numpy as np

In [ ]:
def SGD_method(weight, f_input, correct_output):
# weight:           vettore dei pesi relativi alle feature
# f_input:          dataset contenente i valori delle feature per ciascun campione
# correct_output:   vettore dei valori attesi per ciascun campione

    LR = 0.9    # learning rate
    N  = 4      # numero di epoche

    for k in range(1, N+1):                                 # ciclo sui campioni
        transposed_input = f_input[k, :]                    # prendo i valori delle feature relativi al campione k-esimo
        expected = correct_output[k]                        # prendo il valore atteso del k-esimo campione
        lin_comb = weight * transposed_input                # combinazione lineare delle feature pesate
        predicted = Sigmoid(lin_comb)                       # normalizzazione della comb. lin. attraverso la funzione Sigmoid         
        cost_function_der = expected - predicted            # derivata della funzione di costo relativa alla metrica Squared Error Cost (dimezzata)                          
        delta = predicted*(1-predicted)*cost_function_der   # derivata di Sigmoid*Cost_function


        # (sigmoid)^2 * e^-x = p^2

        # p*(p-e)^2 = 

In [ ]:
# function Weight = SGD_method(Weight, input, correct_Output)
#                                                                                 %input 3 neurons
                                                                                
#     LR = 0.9;
#     N = 4;
#     for k=1:N
#        transposed_Input = input(k, :)';
#        expected = correct_Output(k);
#        lin_comb = Weight*transposed_Input;
#        predicted = Sigmoid(lin_comb);                                    
#        cost_function_der = expected - predicted;                               % cost_function_der is derivate of cost function
#        delta = predicted*(1-predicted)*cost_function_;                         % delta is derivate of sigmoid*cost_function 
#                                                                                % derivate of the sigmoid is sigmoid*(1-sigmoid)
#        dWeight = LR*delta*transposed_Input;                                    % dWeigth = LR*derivate(cost_function) = LR*(delta*input)
#                                                                                % weight correction given by learning rate
#                                                                                % multiplied by delta and the transposed input
#        Weight(1) = Weight(1) + dWeight(1);
#        Weight(2) = Weight(2) + dWeight(2);
#        Weight(3) = Weight(3) + dWeight(3);
#     end
# end